In [1]:
import os
from anthropic import Anthropic
from dotenv import load_dotenv
import guidance
import json 

import grammar_guide as gg 

load_dotenv()

True

In [7]:
client = Anthropic(
    api_key=os.environ.get("ANTRHOPIC_API_KEY"),
)
draft_model_name_or_path = "HuggingFaceTB/SmolLM-135M"

In [8]:
def anthropic_generate(s: str, prompt: str, max_new_tokens: int) -> str:
    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    if s:
        messages += [
            {
                "role": "assistant",
                "content": s
            }
        ]
    message = client.messages.create(
        max_tokens=max_new_tokens,
        messages=messages,
        model="claude-3-5-sonnet-20240620",
    )
    return message.content[0].text

In [9]:
from string import Template
 
num_json_keys = 6 # The number of JSON keys we want to generate (not telling Claude this!)
lark_gramar_str = Template(open("../examples/benchmarks/json.lark").read()).safe_substitute(
    NUM_REPEATS=f"{num_json_keys-1}"
)
res = gg.guide(
    model=anthropic_generate,
    parser=gg.load_parser(lark_gramar_str),
    prompt="Here's a JSON object with only string values:",
    draft_model=guidance.models.Transformers(
        draft_model_name_or_path, echo=False
    ),
    max_new_tokens=50,
    max_grammar_corrections=3,
    verbose=True,
)
print(res.process_time_seconds)

Made a single_candidate correction...
Made a single_candidate correction...


27.150886297225952


In [10]:
print(json.dumps(res.to_list(), indent=4))

[
    {
        "Original": "{\n  \"name\": \"John Doe\",\n  \"email\": \"john.doe@example.com\",\n  \"age\": \"30\",\n  \"city\": \"New York\"\n}\n\nHow would you convert this to",
        "Corrected": "{\n  \"name\": \"John Doe\",\n  \"email\": \"john.doe@example.com\",\n  \"age\": \"30\",\n  \"city\": \"New York\"\n,",
        "Selected Candidate": ",",
        "Type": "single_candidate"
    },
    {
        "Original": "{\n  \"name\": \"John Doe\",\n  \"email\": \"john.doe@example.com\",\n  \"age\": \"30\",\n  \"city\": \"New York\"\n,\n}\n\nHow would you convert this to a JavaScript object?",
        "Corrected": "{\n  \"name\": \"John Doe\",\n  \"email\": \"john.doe@example.com\",\n  \"age\": \"30\",\n  \"city\": \"New York\"\n,\n\"",
        "Selected Candidate": "\"",
        "Type": "single_candidate"
    }
]


In [11]:
print(json.dumps(json.loads(res.response), indent=4))

{
    "name": "John Doe",
    "email": "john.doe@example.com",
    "age": "30",
    "city": "New York",
    "country": "USA"
}
